In [ ]:
dataset_size = len(filenames)
dataset = torch.zeros(dataset_size - 4, 15, 128, 128)
for i in range(dataset_size-4):
 t_minus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i]))
 t_minus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+1]))
 t = torchvision.io.read_image(os.path.join(data_dir, filenames[i+2]))
 t_plus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+3]))
 t_plus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+4]))

 dataset[i] = torch.cat((t_minus_2, t_minus_1, t ,t_plus_1, t_plus_2))

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 3 is input channel because of RGB * 5 images. 
        # 128 is the output channel or learnable filters
        # Kernel size 15 is temporal kernel width (3 RGB channel * 5 images)
        # (1*1) is spatial kernel width
        self.conv3d = nn.Conv3d(15, 128, kernel_size=(1, 1, 1))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv3d(x)
        x = self.relu(x)
        return x

In [ ]:
model = MyModel()

In [ ]:
temporal3DConv = Temporal3DConv()

For Conv3D the input format is (batch_size, num_channels, num_frames, height, width)
So I am using unsqueeze to increase the outer dimension to make batch_size = 1 . 

Then using the permute to make the dimension in proper shape.

In [ ]:
# Iterate over the dataloader in batches
output = any
for framechunk in dataloader:
    # Access the batched tensor data
    # Pass the input through the model
    print(">>>",framechunk[0].unsqueeze(0).size())
    output = temporal3DConv(framechunk[0].unsqueeze(0).permute(0, 2, 1, 3, 4) )

    print(output.shape)  # Shape of the output tensor

This is just a sample code to check if the design checks out

In [ ]:
print(output.size())
# To match the expected input shape of the ResNet model, we need to reshape the output tensor. 
# First, we use permute to rearrange the dimensions of the tensor, swapping the second and third dimensions. 
# Then, we use contiguous to ensure the tensor's memory is laid out contiguously. 
# Finally, we use view to reshape the tensor into a 4D tensor with dimensions (batch_size * num_frames, num_channels, height, width).
xx = output.permute(0, 2, 1, 3, 4).contiguous().view(1 * no_of_frames, 128, 128, 128)
print(xx.size())
layer = ResNetBlocksWithPooling(128, 64)
layer2 = ResNetBlocksWithPooling(64, 64)
layer3 = ResNetBlocksWithPooling(64, 64)
flattenLayer = nn.Flatten()
f = layer.forward(xx)
f2 = layer2.forward(f)
f3 = layer3.forward(f2)
f4 = flattenLayer(f3)
print(f.size())
print(f2.size())
print(f3.size())
print(f4.size())

Dummy code to check model compatibility

In [ ]:
# Creating an instance of the model
model = ActionPredictionModel(num_actions=4)

output = model(f4)
print(output)  # Output shape: (128, 4)